# Dependencies / Setup

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json
import datetime

# Today's Date
now = datetime.datetime.now()

# PyMySQL
import pymysql
pymysql.install_as_MySQLdb()

# Import API key
from config import api_key

timetag = now.strftime("%Y_%m_%d_%H%M")
print(timetag)

2018_12_08_1804


## Import video games list from kaggle source

In [13]:
csv_file = "vgsales.csv"
vgsales_data_df = pd.read_csv(csv_file)
vgsales_data_df.head(1000)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
5,6,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26
6,7,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,8,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
8,9,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.59,7.06,4.70,2.26,28.62
9,10,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31


In [3]:
#Mask for specific rank (or # in list)
#selected 10 to start with to try to get API pull to work
#My goal is to get records 5501 through 11001 on the list
start_rank = 5501
end_rank = 11001
mask = (vgsales_data_df['Rank'] >= start_rank) & (vgsales_data_df['Rank'] <= end_rank)

#ESRB, total_rating, popularity, hypes
names_list = vgsales_data_df.loc[mask]["Name"].values.tolist()
names_list

['Deus Ex: Mankind Divided',
 'Cool Boarders 2001',
 'Classic NES Series: Castlevania',
 'Dragon Ball Z: Battle of Z',
 'Scooby-Doo! First Frights',
 'Fight Night Round 3',
 'GT Advance Championship Racing',
 'Way of the Samurai 2',
 'FIFA Street 3',
 'Custom Robo',
 'BattleTanx',
 'Hatsune Miku: Project Diva Extend',
 'Battlestations: Pacific',
 'Pitfall 3D: Beyond the Jungle',
 'Disgaea 2: Cursed Memories',
 "Cabela's Dangerous Hunts 2011",
 'DECA Sports Freedom',
 'Clock Tower 3',
 'Battle Dodge Ball',
 'Goosebumps HorrorLand',
 'Phineas and Ferb: Across the 2nd Dimension',
 'Call Of Duty 2: Big Red One',
 'Elebits',
 'Warhammer 40,000: Squad Command',
 'F1 2013',
 'Disgaea: Hour of Darkness',
 'NeoGeo Battle Coliseum',
 'SSX Blur',
 'DiRT 2',
 'Vin Diesel: Wheelman',
 'SpongeBob SquarePants: Creature from the Krusty Krab',
 'My Baby: First Steps',
 'MLB 08: The Show',
 'The Black Eyed Peas Experience',
 'Parodius',
 'Spy Kids 3-D: Game Over',
 'Sakura Wars 3: Paris wa Moeteiru ka',

### Perform API Calls
* Perform an ESRB check on each name using a series of successive API calls.
* Include a print log of each video game as it is being processed (with the game name and esrb rating).


In [4]:
#API config data
# Save config information
url = "https://api-endpoint.igdb.com"
headers = {"user-key" : api_key,"Accept" : "application/json"}
# Build partial query URL
query_url = f"{url}/games/?search="

In [5]:
#ESRB, total_rating, popularity, hypes

# set up lists to hold reponse info
title = []
ESRB = []
total_rating = []
popularity = []
hypes = []

# Creates the count variable for record keeping
count = 1

# Loop through the list of cities and perform a request for data on each
print("Beginning Data Retrieval")
print("------------------------")
for name in names_list:
    try:
        print(f"Processing record {count} of {len(names_list)} | {name}")
        print(f"{query_url}{name}")
        response = requests.get(query_url + name+"&fields=name,esrb,total_rating,popularity", headers=headers).json()
        
        #Append information to lists
        try:
            title.append(response[0]["name"])
        except KeyError:
            title.append("NaN")
            continue
        try:
            ESRB.append(response[0]["esrb"])
        except KeyError:
            ESRB.append("NaN")
            continue
        try:
            total_rating.append(response[0]["total_rating"])
        except KeyError:
            total_rating.append("NaN")
            continue
        try:
            popularity.append(response[0]["popularity"])
        except KeyError:
            popularity.append("NaN")
            continue
        try:
            hypes.append(response[0]["hypes"])
        except KeyError:
            hypes.append("NaN")
    
    except:
        print("Game not found, skipped.")
        
    finally:
        if count < 6000:
            count += 1
        else:
            break

Beginning Data Retrieval
------------------------
Processing record 1 of 5501 | Deus Ex: Mankind Divided
https://api-endpoint.igdb.com/games/?search=Deus Ex: Mankind Divided
Processing record 2 of 5501 | Cool Boarders 2001
https://api-endpoint.igdb.com/games/?search=Cool Boarders 2001
Processing record 3 of 5501 | Classic NES Series: Castlevania
https://api-endpoint.igdb.com/games/?search=Classic NES Series: Castlevania
Processing record 4 of 5501 | Dragon Ball Z: Battle of Z
https://api-endpoint.igdb.com/games/?search=Dragon Ball Z: Battle of Z
Processing record 5 of 5501 | Scooby-Doo! First Frights
https://api-endpoint.igdb.com/games/?search=Scooby-Doo! First Frights
Processing record 6 of 5501 | Fight Night Round 3
https://api-endpoint.igdb.com/games/?search=Fight Night Round 3
Processing record 7 of 5501 | GT Advance Championship Racing
https://api-endpoint.igdb.com/games/?search=GT Advance Championship Racing
Processing record 8 of 5501 | Way of the Samurai 2
https://api-endpoint.

Processing record 67 of 5501 | Batman: Arkham Origins
https://api-endpoint.igdb.com/games/?search=Batman: Arkham Origins
Processing record 68 of 5501 | Discovery Kids: Puppy Playtime
https://api-endpoint.igdb.com/games/?search=Discovery Kids: Puppy Playtime
Processing record 69 of 5501 | Dr. Mario / Puzzle League
https://api-endpoint.igdb.com/games/?search=Dr. Mario / Puzzle League
Processing record 70 of 5501 | Bleach: Shattered Blade
https://api-endpoint.igdb.com/games/?search=Bleach: Shattered Blade
Processing record 71 of 5501 | Shin Nippon Pro Wrestling: Toukon Retsuden
https://api-endpoint.igdb.com/games/?search=Shin Nippon Pro Wrestling: Toukon Retsuden
Processing record 72 of 5501 | Major League Baseball 2K6
https://api-endpoint.igdb.com/games/?search=Major League Baseball 2K6
Processing record 73 of 5501 | Pinball Hall of Fame: The Gottlieb Collection
https://api-endpoint.igdb.com/games/?search=Pinball Hall of Fame: The Gottlieb Collection
Processing record 74 of 5501 | God of

Processing record 135 of 5501 | DiRT Rally
https://api-endpoint.igdb.com/games/?search=DiRT Rally
Processing record 136 of 5501 | Fortress
https://api-endpoint.igdb.com/games/?search=Fortress
Processing record 137 of 5501 | FIFA 2001 Major League Soccer
https://api-endpoint.igdb.com/games/?search=FIFA 2001 Major League Soccer
Processing record 138 of 5501 | Peppa Pig: Theme Park Fun
https://api-endpoint.igdb.com/games/?search=Peppa Pig: Theme Park Fun
Processing record 139 of 5501 | Rally Cross
https://api-endpoint.igdb.com/games/?search=Rally Cross
Processing record 140 of 5501 | My Horse & Me
https://api-endpoint.igdb.com/games/?search=My Horse & Me
Processing record 141 of 5501 | Frogger's Adventures: The Rescue
https://api-endpoint.igdb.com/games/?search=Frogger's Adventures: The Rescue
Processing record 142 of 5501 | Extreme Pinball
https://api-endpoint.igdb.com/games/?search=Extreme Pinball
Processing record 143 of 5501 | Shaun White Snowboarding
https://api-endpoint.igdb.com/gam

Processing record 200 of 5501 | Fable III
https://api-endpoint.igdb.com/games/?search=Fable III
Processing record 201 of 5501 | Dragon Ball: XenoVerse
https://api-endpoint.igdb.com/games/?search=Dragon Ball: XenoVerse
Processing record 202 of 5501 | Might & Magic Heroes VI
https://api-endpoint.igdb.com/games/?search=Might & Magic Heroes VI
Processing record 203 of 5501 | Fuel
https://api-endpoint.igdb.com/games/?search=Fuel
Processing record 204 of 5501 | ExerBeat
https://api-endpoint.igdb.com/games/?search=ExerBeat
Processing record 205 of 5501 | WarCraft II: The Dark Saga
https://api-endpoint.igdb.com/games/?search=WarCraft II: The Dark Saga
Processing record 206 of 5501 | MonHun Nikki: Poka Poka Ailu Mura G
https://api-endpoint.igdb.com/games/?search=MonHun Nikki: Poka Poka Ailu Mura G
Processing record 207 of 5501 | Crash Nitro Kart
https://api-endpoint.igdb.com/games/?search=Crash Nitro Kart
Processing record 208 of 5501 | Sesame Street: Cookie's Counting Carnival
https://api-endp

Processing record 265 of 5501 | Taiko Drum Master: Don and Katsu's Space-Time Great Adventure
https://api-endpoint.igdb.com/games/?search=Taiko Drum Master: Don and Katsu's Space-Time Great Adventure
Processing record 266 of 5501 | Calvin Tucker's Redneck Jamboree
https://api-endpoint.igdb.com/games/?search=Calvin Tucker's Redneck Jamboree
Processing record 267 of 5501 | MySims SkyHeroes
https://api-endpoint.igdb.com/games/?search=MySims SkyHeroes
Processing record 268 of 5501 | J-League Winning Eleven 2007: Club Championship
https://api-endpoint.igdb.com/games/?search=J-League Winning Eleven 2007: Club Championship
Processing record 269 of 5501 | Shrek's Carnival Craze Party Games
https://api-endpoint.igdb.com/games/?search=Shrek's Carnival Craze Party Games
Processing record 270 of 5501 | Harvest Moon: Magical Melody
https://api-endpoint.igdb.com/games/?search=Harvest Moon: Magical Melody
Processing record 271 of 5501 | Star Wars Jedi Knight II: Jedi Outcast
https://api-endpoint.igdb

...

Game not found, skipped.
Processing record 5228 of 5501 | Ultra Street Fighter IV
https://api-endpoint.igdb.com/games/?search=Ultra Street Fighter IV
Game not found, skipped.
Processing record 5229 of 5501 | Animal Planet: Vet Life
https://api-endpoint.igdb.com/games/?search=Animal Planet: Vet Life
Game not found, skipped.
Processing record 5230 of 5501 | Cubic Ninja
https://api-endpoint.igdb.com/games/?search=Cubic Ninja
Game not found, skipped.
Processing record 5231 of 5501 | Championship Pony
https://api-endpoint.igdb.com/games/?search=Championship Pony
Game not found, skipped.
Processing record 5232 of 5501 | Harvest Moon: Frantic Farming
https://api-endpoint.igdb.com/games/?search=Harvest Moon: Frantic Farming
Game not found, skipped.
Processing record 5233 of 5501 | Rise of the Tomb Raider
https://api-endpoint.igdb.com/games/?search=Rise of the Tomb Raider
Game not found, skipped.
Processing record 5234 of 5501 | Pinball Hall of Fame: The Williams Collection
https://api-endpoint

Game not found, skipped.
Processing record 5284 of 5501 | Touch Mechanic
https://api-endpoint.igdb.com/games/?search=Touch Mechanic
Game not found, skipped.
Processing record 5285 of 5501 | Ciel Nosurge: Ushinawareta Hoshi e Sasagu Uta
https://api-endpoint.igdb.com/games/?search=Ciel Nosurge: Ushinawareta Hoshi e Sasagu Uta
Game not found, skipped.
Processing record 5286 of 5501 | Ar tonelico: Melody of Elemia
https://api-endpoint.igdb.com/games/?search=Ar tonelico: Melody of Elemia
Game not found, skipped.
Processing record 5287 of 5501 | Man vs. Wild
https://api-endpoint.igdb.com/games/?search=Man vs. Wild
Game not found, skipped.
Processing record 5288 of 5501 | Sol Trigger
https://api-endpoint.igdb.com/games/?search=Sol Trigger
Game not found, skipped.
Processing record 5289 of 5501 | Pony Friends 2
https://api-endpoint.igdb.com/games/?search=Pony Friends 2
Game not found, skipped.
Processing record 5290 of 5501 | Castlevania Double Pack
https://api-endpoint.igdb.com/games/?search=

Game not found, skipped.
Processing record 5338 of 5501 | Angry Birds: Rio
https://api-endpoint.igdb.com/games/?search=Angry Birds: Rio
Game not found, skipped.
Processing record 5339 of 5501 | Dance on Broadway
https://api-endpoint.igdb.com/games/?search=Dance on Broadway
Game not found, skipped.
Processing record 5340 of 5501 | Risen 3: Titan Lords
https://api-endpoint.igdb.com/games/?search=Risen 3: Titan Lords
Game not found, skipped.
Processing record 5341 of 5501 | Dungeons
https://api-endpoint.igdb.com/games/?search=Dungeons
Game not found, skipped.
Processing record 5342 of 5501 | Pac-Man World 3
https://api-endpoint.igdb.com/games/?search=Pac-Man World 3
Game not found, skipped.
Processing record 5343 of 5501 | SingStar Apres-Ski Party 2
https://api-endpoint.igdb.com/games/?search=SingStar Apres-Ski Party 2
Game not found, skipped.
Processing record 5344 of 5501 | Raven Squad: Operation Hidden Dagger
https://api-endpoint.igdb.com/games/?search=Raven Squad: Operation Hidden Dag

Game not found, skipped.
Processing record 5395 of 5501 | Uta no Prince-Sama: Debut
https://api-endpoint.igdb.com/games/?search=Uta no Prince-Sama: Debut
Game not found, skipped.
Processing record 5396 of 5501 | Galactic Taz Ball
https://api-endpoint.igdb.com/games/?search=Galactic Taz Ball
Game not found, skipped.
Processing record 5397 of 5501 | Freekstyle
https://api-endpoint.igdb.com/games/?search=Freekstyle
Game not found, skipped.
Processing record 5398 of 5501 | Ford Mustang: The Legend Lives
https://api-endpoint.igdb.com/games/?search=Ford Mustang: The Legend Lives
Game not found, skipped.
Processing record 5399 of 5501 | Virtua Fighter CG Portrait Series Vol.3: Akira Yuki
https://api-endpoint.igdb.com/games/?search=Virtua Fighter CG Portrait Series Vol.3: Akira Yuki
Game not found, skipped.
Processing record 5400 of 5501 | Digimon Rumble Arena 2
https://api-endpoint.igdb.com/games/?search=Digimon Rumble Arena 2
Game not found, skipped.
Processing record 5401 of 5501 | Scooby-D

Game not found, skipped.
Processing record 5448 of 5501 | Tak: The Great Juju Challenge
https://api-endpoint.igdb.com/games/?search=Tak: The Great Juju Challenge
Game not found, skipped.
Processing record 5449 of 5501 | NBA 2K16
https://api-endpoint.igdb.com/games/?search=NBA 2K16
Game not found, skipped.
Processing record 5450 of 5501 | Def Jam Fight For NY: The Takeover
https://api-endpoint.igdb.com/games/?search=Def Jam Fight For NY: The Takeover
Game not found, skipped.
Processing record 5451 of 5501 | NHL 2K8
https://api-endpoint.igdb.com/games/?search=NHL 2K8
Game not found, skipped.
Processing record 5452 of 5501 | Black & White 2
https://api-endpoint.igdb.com/games/?search=Black & White 2
Game not found, skipped.
Processing record 5453 of 5501 | GripShift
https://api-endpoint.igdb.com/games/?search=GripShift
Game not found, skipped.
Processing record 5454 of 5501 | Jenga World Tour
https://api-endpoint.igdb.com/games/?search=Jenga World Tour
Game not found, skipped.
Processing 

### Save Data

In [15]:
#Create a dictionary of lists
games = {"name":title,"esrb":ESRB,"popularity":popularity,"hypes":hypes,"total_rating":total_rating}
print(games)

{'name': ['Deus Ex: Mankind Divided', 'Cool Boarders 2001', 'Classic NES Series: Castlevania', 'Dragon Ball Z: Battle of Z', 'Scooby-Doo! First Frights', 'Fight Night Round 3', 'GT Advance Championship Racing', 'Way of the Samurai 2', 'FIFA Street 3', 'Custom Robo', 'BattleTanx', 'Hatsune Miku: Project DIVA Extend', 'Battlestations: Pacific', 'Pitfall 3D: Beyond the Jungle', 'Disgaea 2: Cursed Memories', "Cabela's Dangerous Hunts 2011", 'Deca Sports Freedom', 'Clock Tower 3', 'Battle Dodge Ball', 'Goosebumps: HorrorLand', 'Phineas and Ferb: Across the 2nd Dimension', 'Call of Duty 2: Big Red One', 'Elebits', 'Warhammer 40,000: Squad Command', 'F1 2013', 'Disgaea: Hour of Darkness', 'NeoGeo Battle Coliseum', 'SSX Blur', 'DiRT 4', 'Vin Diesel Wheelman', 'Spongebob Squarepants: Creature From the Krusty Krab', 'My Baby First Steps', 'MLB 08: The Show', 'The Black Eyed Peas Experience', 'Parodius', 'Spy Kids 3-D: Game Over', 'Sakura Taisen 3: Pari wa Moete iru ka?', 'Picross Touch', 'NCAA G

In [18]:
#Create a Pandas DataFrame
# create dataframe oriented by index first since lists are not the same length
games_df = pd.DataFrame.from_dict(games, orient='index')
#Transpose the dataframe
games_df = games_df.transpose()
#Preview
games_df.head()

,name,esrb,popularity,hypes,total_rating
0,Deus Ex: Mankind Divided,{'rating': 6},24.3333,NaN,79.6174
1,Cool Boarders 2001,{'rating': 3},7.33333,NaN,NaN
2,Classic NES Series: Castlevania,NaN,1.33333,NaN,75.8125
3,Dragon Ball Z: Battle of Z,{'synopsis': 'his is a fighting game in which ...,3.33333,NaN,58.3333
4,Scooby-Doo! First Frights,"{'synopsis': '', 'rating': 4}",1.33333,NaN,75.7613


In [17]:
#Output DataFrame as a csv
games_df.to_csv("Output_EFB/games_df.csv", encoding = "utf-8", index = False)

In [22]:
#Split the esrb column
esrb_split_df = games_df['esrb'].apply(pd.Series)
esrb_split_df.head()

C:\Users\maden\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexes\api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
C:\Users\maden\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


,rating,0,synopsis
0,6.0,NaN,NaN
1,3.0,NaN,NaN
2,NaN,NaN,NaN
3,5.0,NaN,his is a fighting game in which players contro...
4,4.0,NaN,


In [42]:
#Concat original df and split esrb df
split_games_df = pd.concat([games_df.drop(['esrb'], axis = 1), esrb_split_df],axis=1)
split_games_df.rename(columns = {'rating':'esrb'}, inplace = True)
split_games_df.drop(['hypes','synopsis',0], axis = 1, inplace = True)
split_games_df.head(100)

,name,popularity,total_rating,esrb
0,Deus Ex: Mankind Divided,24.3333,79.6174,6.0
1,Cool Boarders 2001,7.33333,NaN,3.0
2,Classic NES Series: Castlevania,1.33333,75.8125,NaN
3,Dragon Ball Z: Battle of Z,3.33333,58.3333,5.0
4,Scooby-Doo! First Frights,1.33333,75.7613,4.0
5,Fight Night Round 3,1.66667,70,5.0
6,GT Advance Championship Racing,1.66667,60.8564,3.0
7,Way of the Samurai 2,1.33333,70.1955,6.0
8,FIFA Street 3,1.33333,74.6818,3.0
9,Custom Robo,2.33333,NaN,5.0


In [41]:
split_games_df.dtypes

name             object
popularity       object
total_rating     object
esrb            float64
dtype: object

In [44]:
#Output Cleanee DataFrame as a csv
split_games_df.to_csv("Output_EFB/games_df_clean.csv", encoding = "utf-8", index = False)